# Giving the AI Tools: Interacting with External Information and Actions

#### Generative AI models are powerful, but they have limitations. 
* They have a knowledge cut-off (they don't know about the absolute latest events)
* They can't perform complex calculations perfectly
* They can't directly interact with external systems like databases or APIs.

#### This is where Tools come in. By defining specific functions (tools) that your code can execute, you can empower the AI to:
* Access up-to-date information: Look up current weather, stock prices, news, definitions, etc.
* Perform accurate calculations: Use a calculator for reliable math.
* Interact with services: Send emails, set reminders, query databases (with appropriate setup).
* Retrieve specific knowledge: Look up information in internal documents or databases (a common pattern called RAG).

In [1]:
import google.generativeai as genai

genai.configure(api_key="api_key")

#### Display Response

In [15]:
import json # Import json for pretty printing function call arguments

def display_response(response: genai.types.GenerateContentResponse):
    if response is None:
        print("Received an empty or None response.")
        return

    # A response can contain one or more 'parts'.
    # These parts can be text, function calls, etc.
    if response.parts:
        print("Content Parts:")
        for i, part in enumerate(response.parts):
            print(f"  Part {i+1}:")
            if part.text:
                print(f"    Type: Text")
                print("    Content:")
                print("      " + part.text.replace('\n', '\n      '))
            elif part.function_call:
                print(f"    Type: Function Call Request")
                print(f"    Name: {part.function_call.name}")
            else:
                print(f"    Type: Unknown/Other Part Type")
                print(f"    Raw Part Data: {part}")

    elif not response.parts:
         print("Response contains no content parts.")
         if response.prompt_feedback:
             print("Prompt Feedback:")
             print(response.prompt_feedback)

    print("----------------------\n")

#### Define your tools

In [2]:
import google.generativeai.types as genai_types # Need this for ToolConfig
import os


weather_tool = genai.protos.Tool(
    function_declarations=[
        genai.protos.FunctionDeclaration(
            name='get_weather',
            description='Gets the current weather conditions for a specific location.',
            parameters=genai.protos.Schema(
                type="OBJECT", 
                properties={
                    'location': genai.protos.Schema(type="STRING", description='The city and state or country, e.g. "San Francisco, CA" or "London, UK"'), 
                },
                required=['location'],
            ),
        )
    ]
)

definition_tool = genai.protos.Tool(
    function_declarations=[
        genai.protos.FunctionDeclaration(
            name='lookup_definition',
            description='Looks up the definition of a given word.',
            parameters=genai.protos.Schema(
                type="OBJECT", 
                properties={
                    'word': genai.protos.Schema(type="STRING", description='The word to look up.'), 
                },
                required=['word'],
            ),
        )
    ]
)

# Combine the tools you want this model instance to use
available_tools = [weather_tool, definition_tool]

#### Initialize the model *with* the tools

In [3]:
model = genai.GenerativeModel(
    'models/gemini-2.0-flash',
    tools=available_tools, 
)

print("Model initialized with tools.")

Model initialized with tools.


#### Helper function to execute tool calls (basic simulation for demo)
In a real application, this would be more robust and handle errors

In [4]:
def execute_tool_call(function_call):
    tool_name = function_call.name
    args = function_call.args

    print(f"Executing tool: {tool_name} with args: {args}")

    # --- Define your tool execution logic here ---
    if tool_name == "get_weather":
        location = args.get("location", "unknown")
        # Simulate an API call - in reality, call a weather API
        if "london" in location.lower():
            return {"temperature": "15°C", "conditions": "Cloudy"}
        elif "new york" in location.lower():
             return {"temperature": "22°C", "conditions": "Sunny"}
        else:
             return {"temperature": "N/A", "conditions": "Location not found"}
    elif tool_name == "lookup_definition":
         word = args.get("word", "")
         # Simulate a dictionary lookup
         definitions = {
             "ephemeral": "lasting for a very short time.",
             "ubiquitous": "present, appearing, or found everywhere.",
             "serendipity": "the occurrence and development of events by chance in a happy or beneficial way."
         }
         return {"definition": definitions.get(word.lower(), f"Definition not found for '{word}'")}
    # ----------------------------------------------

    return {"error": f"Tool '{tool_name}' not implemented."}

#### Use the initialized model to start the chat

In [14]:
convo = model.start_chat()

user_query = "What is the weather in London?"

print(f"Sending query: '{user_query}'")

# Send the user's query
response = convo.send_message(user_query)

# Display the immediate response (should be a tool call)
display_response(response)

# The model predicts the need for a tool and predicts the structure of the function call, 
#  but it cannot run the function itself.
if response.parts and response.parts[0].function_call:
    function_call = response.parts[0].function_call

    # Execute the function call using the helper function
    tool_result = execute_tool_call(function_call)

    print(f"Tool execution result: {tool_result}")

    print("Sending tool result back to the model...")
    response_after_tool = convo.send_message(genai.protos.Content(
        role='function',
        parts=[genai.protos.Part(function_response=genai.protos.FunctionResponse(
            name=function_call.name,
            response=tool_result
        ))]
    ))
    
    # print(f"response_after_tool: {response_after_tool}")
    # Display the final response
    display_response(response_after_tool)
else:
    print("Model did not request a tool.") 

Sending query: 'What is the weather in London?'
Content Parts:
  Part 1:
    Type: Function Call Request
    Name: get_weather
----------------------

Executing tool: get_weather with args: <proto.marshal.collections.maps.MapComposite object at 0x107d47310>
Tool execution result: {'temperature': '15°C', 'conditions': 'Cloudy'}
Sending tool result back to the model...
Content Parts:
  Part 1:
    Type: Text
    Content:
      The weather in London, UK is cloudy with a temperature of 15°C.
      
----------------------

